In [ ]:
!nvidia-smi

In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import pandas as pd
import numpy as np
import re
import librosa
import seaborn as sns
import os
import json
import IPython.display as ipd
import soundfile as sf
import torch
import h5py
import onnxruntime as ort
import openvino as ov
import tensorflow_hub as hub
import torch.quantization.quantize_fx as quantize_fx

from glob import glob
from tqdm import tqdm
from pprint import pprint
from matplotlib import pyplot as plt
from itertools import chain
from os.path import join as pjoin
from torchaudio.transforms import AmplitudeToDB, MelSpectrogram
from copy import deepcopy
from sklearn.metrics import f1_score 
from scipy.special import expit

# from code_base.utils import parallel_librosa_load, groupby_np_array, stack_and_max_by_samples, macro_f1_similarity, N_CLASSES_2021_2022, N_CLASSES_2021, comp_metric, N_CLASSES_XC_LIGIT_SHORTEN, N_CLASSES_XC_LIGIT_EVEN_SHORTEN
# from code_base.utils.constants import SAMPLE_RATE
from code_base.utils.onnx_utils import ONNXEnsemble, convert_to_onnx
from code_base.models import WaveCNNClasifier, WaveCNNAttenClasifier, WaveTDNNClasifier
from code_base.datasets import WaveDataset, WaveAllFileDataset
from code_base.utils.swa import avarage_weights, delete_prefix_from_chkp
from code_base.inefernce import BirdsInference
from code_base.utils import load_json, compose_submission_dataframe, groupby_np_array, stack_and_max_by_samples, write_json
from code_base.utils.metrics import score_numpy
%matplotlib inline


2024-04-28 13:54:05.042072: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-28 13:54:06.023736: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/backup/vova/src/exps/bird_clef_2024/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


`speechbrain` was not imported
`LEAF` was not imported
colorednoise package is missing


/backup/vova/src/exps/bird_clef_2024/.venv/lib/python3.11/site-packages/nnAudio/Spectrogram.py:4: Warning: importing Spectrogram subpackage will be deprecated soon. You should import the feature extractor from the feature subpackage. See actual documentation.
  warnings.warn(


# Config

In [2]:
MODEL_PATH = "https://www.kaggle.com/models/google/bird-vocalization-classifier/TensorFlow2/bird-vocalization-classifier/8"

In [3]:
bird2id_source = {
    v:k for k,v in pd.read_csv(hub.resolve(MODEL_PATH) + "/assets/label.csv")["ebird2021"].to_dict().items()
}
bird2id_source["bkrfla1"] = bird2id_source.pop("bkrfla2")
bird2id_source["indrol2"] = bird2id_source.pop("indrol1")

bird2id_target = load_json("/home/vova/data/exps/birdclef_2024/class_mappings/bird2int_2024.json")


id2bird_source = {v:k for k,v in bird2id_source.items()}
id2bird_target = {v:k for k,v in bird2id_target.items()}

REARRANGE_INDICES = np.array([
    bird2id_source[id2bird_target[i]] for i in range(len(id2bird_target)) #if id2bird_target[i] not in ('bkrfla1', 'indrol2')
]).astype(int)
REARRANGE_INDICES

array([ 7717,  4041,   354,  4914,  1643,  5186,  4749,  4621,  2750,
        1017,  1802,  4524,  3890,  7289,  8703,  6664,  7355,  9680,
       10237,  1372,  4096,  5373,  6389,  5418,  2593,  4402,  4003,
        6925,  3922, 10533,  7725,  1507,  5486,  8873,  7804,  4250,
        5572,   901,  3624,  1031,  2707,  4791,  4609,  1692,  1016,
        5120,   971, 10611,  2881,  3825,  6772,  4489,  1297,  1661,
        7352,  9047,  2607,  1788,  8236,  6933,  4728,  6906,  5039,
        4579,  2623,  5385, 10065,  7167,   967, 10050,  4522,  7359,
        4754,  4942,  7436,  2598,  5235,  2601,   727,  3913,  6921,
        8530,  6725,  5975,  8832,  4611,  5047,  4174,  9961,   990,
        8029,  6260,  1594,  6649,  3792,  3462,  9378,  5145,  4076,
        1964,  6922,  4623,  4602,  2524,  6315,  2815,  7666,  5718,
        9836,  4753,   362,  5982,  2103,  9888,  2898,  9422,  6268,
        5148,  1600,  7998,   577,  2627, 10584,  5409,  4618,  4794,
        8784,  3697,

In [4]:
CONFIG = {
    # Main
    "run_validation": True,
    "run_test": True,
    # Inference Class
    "use_sigmoid": False,
    "use_compiled_fp16": False,
    "aggregate_preds": True,
    # Data config
    "train_df_path": "/home/vova/data/exps/birdclef_2024/birdclef_2024/train_metadata_extended_noduplv1.csv",
    "split_path": "/home/vova/data/exps/birdclef_2024/cv_splits/birdclef_2024_5_folds_split_nodupl.npy",
    "train_data_root":"/home/vova/data/exps/birdclef_2024/birdclef_2024/train_audio/",
    "test_data_root":"/home/vova/data/exps/birdclef_2024/birdclef_2024/unlabeled_soundscapes/*.ogg",
    "label_map_data_path": "/home/vova/data/exps/birdclef_2024/class_mappings/bird2int_2024.json",
    "scored_birds_path":"/home/vova/data/exps/birdclef_2024/scored_birds/sb_2024.json", 
    "lookback":None,
    "lookahead":None,
    "segment_len":5,
    "step": None,
    "late_normalize": False,
    
    "model_output_key": None,
}   

# Data

In [5]:
bird2id = load_json(CONFIG["label_map_data_path"])

In [6]:
if CONFIG["run_validation"]:
    val_df = [pd.read_csv(CONFIG["train_df_path"])]

In [7]:
if CONFIG["run_test"]:
    test_au_pathes = glob(CONFIG["test_data_root"])[:20]

    test_df = pd.DataFrame({
        "filename": test_au_pathes,
        "duration_s": [librosa.get_duration(filename=el) for el in test_au_pathes]
    })

/tmp/ipykernel_2934077/715364531.py:6: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  "duration_s": [librosa.get_duration(filename=el) for el in test_au_pathes]


In [8]:
if CONFIG["run_validation"]:
    val_ds_conig = {
       "root": CONFIG["train_data_root"],
       "label_str2int_mapping_path": CONFIG["label_map_data_path"],
       "use_audio_cache": True,
       "n_cores": 64,
       "verbose": False,
       "segment_len": CONFIG["segment_len"],
       "lookback":CONFIG["lookback"],
       "lookahead":CONFIG["lookahead"],
       "sample_id": None,
       "late_normalize": CONFIG["late_normalize"],
       "load_normalize": CONFIG.get("load_normalize", False),
       "step": CONFIG["step"],
        "validate_sr": 32_000
    }
if CONFIG["run_test"]:
    ds_config_test = {
       "root": "",
       "label_str2int_mapping_path": CONFIG["label_map_data_path"],
       "n_cores": 64,
       "use_audio_cache": True,
       "test_mode": True,
       "segment_len": CONFIG["segment_len"],
       "lookback":CONFIG["lookback"],
       "lookahead":CONFIG["lookahead"],
        "sample_id": None,
        "late_normalize": CONFIG["late_normalize"],
        "load_normalize": CONFIG.get("load_normalize", False),
        "step": CONFIG["step"],
        "validate_sr": 32_000
    }
loader_config = {
    "batch_size": 64,
    "drop_last": False,
    "shuffle": False,
    "num_workers": 0,
}

In [9]:
if CONFIG["run_test"]:
    ds_test = WaveAllFileDataset(df=test_df, **ds_config_test)
if CONFIG["run_validation"]:
    ds_val = [WaveAllFileDataset(df=df, **val_ds_conig) for df in val_df]

secondary_labels is not found in df. Maybe test or nocall mode


In [10]:
if CONFIG["run_validation"]:
    loader_val = [torch.utils.data.DataLoader(
        ds,
        **loader_config,
    )for ds in ds_val]
if CONFIG["run_test"]:
    loader_test = torch.utils.data.DataLoader(
        ds_test,
        **loader_config,
    )

In [11]:
len(loader_val)

1

# Model

In [12]:
model = hub.load(MODEL_PATH)

2024-04-28 13:54:15.212657: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


# Inference Class

In [13]:
inference_class = BirdsInference(
    device="cpu",
    verbose_tqdm=True,
    use_sigmoid=CONFIG["use_sigmoid"],
    model_output_key=CONFIG["model_output_key"],
    aggregate_preds=CONFIG.get("aggregate_preds", True)
)

In [14]:
inference_class.use_sigmoid

False

# Val Predict

In [ ]:
if CONFIG["run_validation"]:
    val_tgts, val_preds = inference_class.predict_val_loaders(
        nn_models=[model],
        data_loaders=loader_val,
        is_google_model=True,
        google_postprocess=lambda x: expit(x["label"].numpy())[:,REARRANGE_INDICES]
    )
    if CONFIG.get("scored_birds_path", None):
        print("Extracting scored_bird_ids indices")
        scored_bird = load_json(CONFIG["scored_birds_path"])
        scored_bird_ids = [bird2id[el] for el in scored_bird]
        val_tgts = [el[:,scored_bird_ids] for el in val_tgts]
        val_preds = [el[:,scored_bird_ids] for el in val_preds]
    cmaps = [
        score_numpy(gt, pr) for gt, pr in zip(val_tgts, val_preds)
    ]

    print(
        f"Folds Roc Auc: {cmaps}\n"
        f"Mean Roc Auc: {np.mean(cmaps)}"
    )

  0%|                                                                                                                                                                              | 0/3389 [00:00<?, ?it/s]WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
I0000 00:00:1714312470.051506 2934077 service.cc:145] XLA service 0x5aee32298a10 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1714312470.051556 2934077 service.cc:153]   StreamExecutor device (0): Host, Default Version
2024-04-28 13:54:30.322634: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
W0000 00:00:1714312470.332707 2934077 assert_op.cc:38] Ignoring Assert operator jax2tf_infer_fn_/assert_equal_1/Assert/AssertGuard/Assert
I0000 00:00:1714312472.745816 2934077 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at

In [ ]:
print(
    f"Folds Roc Auc: {cmaps}\n"
    f"Mean Roc Auc: {np.mean(cmaps)}\n"
    f"OOF Roc Auc: {score_numpy(np.concatenate(val_tgts), np.concatenate(val_preds))}"
)

In [ ]:
val_preds[0].max(axis=1)

```
Full Checkpoint

Folds Roc Auc: [0.9821510023950407]
Mean Roc Auc: 0.9821510023950407
OOF Roc Auc: 0.9821510023950407

Openvino FP16

Folds Roc Auc: [0.9813698940740585]
Mean Roc Auc: 0.9813698940740585
OOF Roc Auc: 0.9813698940740585
```

# Search for Pseudo

In [ ]:
all_val_tgts = np.concatenate(val_tgts)
all_val_preds = np.concatenate(val_preds)

In [ ]:
all_treshes = np.arange(0.05 , 1.0 + 0.05, 0.05)
all_scores = []
for tresh in tqdm(all_treshes):
    all_scores.append(f1_score(
        all_val_tgts,
        all_val_preds > tresh,
        average="macro"
    ))

In [ ]:
best_idx = np.argmax(all_scores)
best_tresh, best_score = all_treshes[best_idx], all_scores[best_idx]

print(f"Best Score = {best_score} and is reached on {best_tresh}")

plt.title("Macro F1 depending on tresh")
plt.plot(all_treshes, all_scores)
plt.show()

# Test Pred

In [ ]:
if CONFIG["run_test"]:
    test_preds, test_preds_long, test_dfidx, test_end = inference_class.predict_test_loader(
        nn_models=model,
        data_loader=loader_test
    )
    test_pred_df = compose_submission_dataframe(
        probs=test_preds,
        dfidxs=test_dfidx,
        end_seconds=test_end,
        filenames=loader_test.dataset.df[loader_test.dataset.name_col].copy(),
        bird2id=bird2id
    )

In [ ]:
# test_pred_df

# ONNX Test Pred

In [ ]:
inference_class_onnx = BirdsInference(
    device="cpu",
    verbose_tqdm=True,
    use_sigmoid=CONFIG["use_sigmoid"],
    model_output_key=CONFIG["model_output_key"],
    use_compiled_fp16=CONFIG["use_compiled_fp16"],
)
# onnx_model = ort.InferenceSession(
#     # f"../logdirs/{CONFIG['exp_name']}/onnx_ensem_2first_folds/model_simpl.onnx"
#     "../logdirs/convnext_small_fb_in22k_ft_in1k_384_Exp_noamp_64bs_5sec_mixupP05_RandomFiltering_balancedSampler_Radamlr1e4_CosBatchLR1e6_Epoch50_SpecAugV1_FocalLoss_Full/onnx_ensem/model_simpl.onnx"
# )
# core = ov.Core()
# model = core.read_model(model=f"../logdirs/convnext_small_fb_in22k_ft_in1k_384_Exp_noamp_64bs_5sec_mixupP05_RandomFiltering_balancedSampler_Radamlr1e4_CosBatchLR1e6_Epoch50_SpecAugV1_FocalLoss_Full/openvino_ensem/model_simpl.xml")
# compiled_model = core.compile_model(model=model, device_name="CPU")

In [ ]:
test_preds_onnx, test_dfidx_onnx, test_end_onnx = inference_class_onnx.predict_test_loader(
    nn_models=compiled_model,
    data_loader=loader_test,
    is_openvino_model=True,
)

In [ ]:
test_pred_df_onnx = compose_submission_dataframe(
    probs=test_preds_onnx,
    dfidxs=test_dfidx_onnx,
    end_seconds=test_end_onnx,
    filenames=loader_test.dataset.df[loader_test.dataset.name_col].copy(),
    bird2id=bird2id
)

In [ ]:
test_pred_df_onnx.iloc[:,1:].values

In [ ]:
test_pred_df_onnx_fp16 = pd.read_csv("temp_fp16.csv")

In [ ]:
test_pred_df_onnx_fp16.iloc[:,1:].values

In [ ]:
np.abs(test_pred_df_onnx_fp16.iloc[:,1:].values - test_pred_df_onnx.iloc[:,1:].values).max()

In [ ]:
# test_pred_df_onnx.to_csv("temp_fp16.csv", index=False)

In [ ]:
plt.hist(test_pred_df_onnx.iloc[:,1:].values.flatten())

In [ ]:
if CONFIG["run_test"]:
    inference_class_onnx = BirdsInference(
        device="cpu",
        verbose_tqdm=True,
        use_sigmoid=CONFIG["use_sigmoid"],
        model_output_key=CONFIG["model_output_key"],
        
    )
    onnx_model = ort.InferenceSession(
        f"../logdirs/{CONFIG['exp_name']}/onnx_ensem_2first_folds/model_simpl.onnx"
    )
    test_preds_onnx, test_preds_long_onnx, test_dfidx_onnx, test_end_onnx = inference_class_onnx.predict_test_loader(
        nn_models=onnx_model,
        data_loader=loader_test,
        is_onnx_model=True
    )
    test_pred_df_onnx = compose_submission_dataframe(
        probs=test_preds_onnx,
        dfidxs=test_dfidx_onnx,
        end_seconds=test_end_onnx,
        filenames=loader_test.dataset.df[loader_test.dataset.name_col].copy(),
        bird2id=bird2id
    )

In [ ]:
test_pred_df_onnx

# Map Predictions

In [ ]:
if CONFIG.get("check_inf_class", False):
    val_ds_check = WaveAllFileDataset(df=val_df[0], **{
           "root": CONFIG["train_data_root"],
           "label_str2int_mapping_path": CONFIG["label_map_data_path"],
           "n_cores": 64,
           "use_audio_cache": True,
           "test_mode": True,
           "segment_len": CONFIG["segment_len"],
           "lookback":CONFIG["lookback"],
           "lookahead":CONFIG["lookahead"],
            "sample_id": None,
            "late_normalize": CONFIG["late_normalize"],
            "step": CONFIG["step"],
        }
    )
    val_loader_check = torch.utils.data.DataLoader(
        val_ds_check,
        **loader_config
    )
    
    test_preds_check, test_preds_long_check, test_dfidx_check, test_end_check = inference_class.predict_test_loader(
        nn_models=model,
        data_loader=val_loader_check
    )
    test_preds_check_grouped = groupby_np_array(
        groupby_f=test_dfidx_check,
        array_to_group=test_preds_check,
        apply_f=stack_and_max_by_samples,
    )
    print(np.allclose(
        test_preds_check_grouped,
        val_preds
    ))